In [77]:
import pandas as pd
import os
import numpy as np
import sys
sys.path.append('./..')
sys.path.append('./../..')
import glob 
import yaml
import pickle
from collections import Counter
from operator import itemgetter

In [235]:
import seaborn as sns
import matplotlib.pyplot as plt
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
loc = './../../generated_data_v1/us_import2'
f_name_train = 'train_data.csv'
f_name_test = 'test_data.csv'
df_train = pd.read_csv(os.path.join(loc,f_name_train),index_col=None)
df_test = pd.read_csv(os.path.join(loc,f_name_test),index_col=None)


In [203]:
domain_dims = None
with open(os.path.join(loc,'domain_dims.pkl'),'rb') as fh:
    domain_dims = pickle.load(fh)
domain_dims

{'Carrier': 701,
 'ConsigneePanjivaID': 8960,
 'PortOfLading': 286,
 'PortOfUnlading': 75,
 'ShipmentDestination': 136,
 'ShipmentOrigin': 126,
 'ShipperPanjivaID': 10661,
 'HSCode': 97}

In [241]:
def create_coocc_matrix(df, col_1, col_2):
    set_elements_1 = set(list(df[col_1]))
    set_elements_2 = set(list(df[col_2]))
    count_1 = len(set_elements_1)
    count_2 = len(set_elements_2)
    coocc = np.zeros([count_1, count_2])
    df = df[[col_1, col_2]]
    new_df = df.groupby([col_1, col_2]).size().reset_index(name='count')

    for _, row in new_df.iterrows():
        i = row[col_1]
        j = row[col_2]
        coocc[i][j] = row['count']

    print('Col 1 & 2', col_1, col_2, coocc.shape, '>>', (count_1, count_2))
    return coocc


'''
Create co-occurrence between entities using training data. 
Returns a dict { Domain1_+_Domain2 : __matrix__ }
Domain1 and Domain2 are sorted lexicographically
'''


def get_coOccMatrix_dict(df, id_col='PanjivaRecordID'):
    columns = list(df.columns)
    columns.remove(id_col)
    columns = list(sorted(columns))
    columnWise_coOccMatrix_dict = {}

    for i in range(len(columns)):
        for j in range(i + 1, len(columns)):
            col_1 = columns[i]
            col_2 = columns[j]
            key = col_1 + '_+_' + col_2
            res = create_coocc_matrix(df, col_1, col_2)
            columnWise_coOccMatrix_dict[key] = res
    return columnWise_coOccMatrix_dict

In [242]:
columnWise_coOccMatrix_dict = get_coOccMatrix_dict(df_train, id_col='PanjivaRecordID')

Col 1 & 2 Carrier ConsigneePanjivaID (701, 8960) >> (701, 8960)
Col 1 & 2 Carrier HSCode (701, 97) >> (701, 97)
Col 1 & 2 Carrier PortOfLading (701, 286) >> (701, 286)
Col 1 & 2 Carrier PortOfUnlading (701, 75) >> (701, 75)
Col 1 & 2 Carrier ShipmentDestination (701, 136) >> (701, 136)
Col 1 & 2 Carrier ShipmentOrigin (701, 126) >> (701, 126)
Col 1 & 2 Carrier ShipperPanjivaID (701, 10661) >> (701, 10661)
Col 1 & 2 ConsigneePanjivaID HSCode (8960, 97) >> (8960, 97)
Col 1 & 2 ConsigneePanjivaID PortOfLading (8960, 286) >> (8960, 286)
Col 1 & 2 ConsigneePanjivaID PortOfUnlading (8960, 75) >> (8960, 75)
Col 1 & 2 ConsigneePanjivaID ShipmentDestination (8960, 136) >> (8960, 136)
Col 1 & 2 ConsigneePanjivaID ShipmentOrigin (8960, 126) >> (8960, 126)
Col 1 & 2 ConsigneePanjivaID ShipperPanjivaID (8960, 10661) >> (8960, 10661)
Col 1 & 2 HSCode PortOfLading (97, 286) >> (97, 286)
Col 1 & 2 HSCode PortOfUnlading (97, 75) >> (97, 75)
Col 1 & 2 HSCode ShipmentDestination (97, 136) >> (97, 136)
Co

In [6]:
domains = list(df_train.columns)
domains

['PanjivaRecordID',
 'Carrier',
 'ConsigneePanjivaID',
 'HSCode',
 'PortOfLading',
 'PortOfUnlading',
 'ShipmentDestination',
 'ShipmentOrigin',
 'ShipperPanjivaID']

##  1. Start by selecting some routes to be actually containing anomalies

In [190]:
kk = df_train.groupby(['PortOfLading','PortOfUnlading',]).size().reset_index(name='count')
# ==================
# Select pairs such that their count in (25,75) percentile
# ==================

lb =  np.percentile(list(kk['count']),20)
ub =  np.percentile(list(kk['count']),80)
kk_1 = kk.loc[(kk['count']>= lb) & (kk['count']<= ub) ]
kk_2 = kk_1.sample(frac=0.20)
kk_2 = kk_2.reset_index(drop=True)
del kk_2['count']
target_PortOfLading_PortOfUnlading = kk


In [207]:
# ===============
# We need list of comapnies trading in these routes
# ===============
pp = df_train.merge(
    target_PortOfLading_PortOfUnlading, 
    on =['PortOfLading','PortOfUnlading'], 
    how ='inner'
)

candidate_Shipper = list(set(pp['ShipperPanjivaID']))
_count = int(0.1 * domain_dims['ShipperPanjivaID'])
target_Shipper = np.random.choice(candidate_Shipper, size = _count,replace=False)
print('Number of interesting shippers ',len(target_Shipper))

Number of interesting shippers  1066


In [208]:
_count = int(0.1 * domain_dims['ConsigneePanjivaID'])
# ==================================
# Now we have the list of shippers
# Consignee who do business with them are actually suspicous
# ==================================
pp_1 = pp.loc[pp['ShipperPanjivaID'].isin(target_Shipper)]
candidate_Shipper = list(set(pp_1['ConsigneePanjivaID']))
target_Consignee = np.random.choice(candidate_Shipper, size = _count,replace=False)
print('Number of interesting consignee ',len(target_Consignee))


Number of interesting consignee  896


## ---------------------------------------------
# Criteria 1
## We define interesting records as ones which satisfy these 2 conditions:
### 1. contains both these comapnies
### 2. contains the route ( 'PortOfLading','PortOfUnlading' )
## ---------------------------------------------

In [222]:
qq = df_train.loc[(df_train['ShipperPanjivaID'].isin(target_Shipper)) & (df_train['ConsigneePanjivaID'].isin(target_Consignee))]
qq_1 = qq.groupby(['ShipmentOrigin','HSCode','ShipmentDestination']).size().reset_index(name='count')
lb =  np.percentile(list(qq_1['count']),10)
ub =  np.percentile(list(qq_1['count']),90)
_count = int(0.2 *len(qq_1))

target_Origin_HSCode_Dest = qq_1.loc[(qq_1['count']>=lb)&(qq_1['count']<=ub)].sample(n=_count)
del target_Origin_HSCode_Dest['count']

In [223]:
target_Origin_HSCode_Dest

,ShipmentOrigin,HSCode,ShipmentDestination
905,82,53,94
416,52,51,78
842,82,46,117
105,6,39,120
572,70,27,120
...,...,...,...
1070,88,7,78
597,76,23,34
1128,89,49,74
22,0,26,129


## --------------------------------------------
# Criteria 2
## We define interesting records as ones which satisfy these 2 conditions:
### 1. contain one of the comapnies
### 2. conatin the triplet (ShipmentOrigin	HSCode	ShipmentDestination)
## --------------------------------------------

In [226]:

a = df_train.merge(target_Origin_HSCode_Dest, on = ['ShipmentOrigin','HSCode','ShipmentDestination'], how='inner')

In [231]:
b = a.loc[a['ConsigneePanjivaID'].isin(target_Consignee)]
_fixed = ['ShipmentOrigin','HSCode','ShipmentDestination','ConsigneePanjivaID']
_perturb = [ _ for _ in list(domain_dims.keys()) if _ not in _fixed]

In [233]:
# ====
# select 2 of _perturb domains
# set them to random options :  such that the row does not occur in train or test
# ====

def generate_by_criteria( row, _fixed, _perturb, co_occurrence_dict):
    p_d = np.random.choice(_perturb,size=2,replace=False)
    
    is_duplicate = True
    while is_duplicate:
        for _dom in _p_d:
            # select reference_domain from _fixed
            _f_d = np.random.choice(_fixed, size=1, replace=False)[0]
            _f_entity = row[_f_d]

            # select entity in _dom such that it does not co-occur with
            _pair = sorted([_dom,_f_d])
            key = '_'.join(_pair)
            _matrix = co_occurrence_dict[key]
            if _pair[0] == _f_d:
                vec = _matrix[_f_entity,:]
            else:
                vec = _matrix[:,_f_entity]
            # Select e such that vec[e] == 0
            pool = list(numpy.where(vec == 0)[0])
            e = np.random.choice(pool,size=1)[0]
            row[_dom] = e  
        # check for duplicates
        
    return row







In [229]:
a.loc[a['ShipperPanjivaID'].isin(target_Shipper)]

,PanjivaRecordID,Carrier,ConsigneePanjivaID,HSCode,PortOfLading,PortOfUnlading,ShipmentDestination,ShipmentOrigin,ShipperPanjivaID
1,107419709,178,3001,4,103,53,85,82,5676
4,107419706,178,3001,4,103,53,85,82,5676
9,106584046,178,3001,4,103,53,85,82,5676
10,107073196,178,3001,4,103,53,85,82,5676
11,107419692,178,3001,4,103,53,85,82,5676
...,...,...,...,...,...,...,...,...,...
15610,113022192,500,8955,22,117,14,53,35,7878
15611,113022193,500,8955,22,117,14,53,35,7878
15612,112684456,227,6650,78,103,69,131,103,1758
15613,113112864,619,3777,27,18,45,120,70,4575


In [236]:
from pandarallel import pandarallel
pandarallel.initialize()

In [237]:
df_size =100
tdf = pd.DataFrame(
    dict(a=np.random.randint(1, 8, df_size),
         b=np.random.rand(df_size)))
tdf

,a,b
0,1,0.272451
1,1,0.243930
2,6,0.184910
3,4,0.757918
4,7,0.891960
...,...,...
95,6,0.181823
96,2,0.423721
97,4,0.946381
98,4,0.610226


In [238]:

def func(row):
    row['a'] = row['a']+10
    return row

In [239]:
res_parallel = tdf.parallel_apply(func, axis=1)

In [240]:
res_parallel

,a,b
0,11.0,0.272451
1,11.0,0.243930
2,16.0,0.184910
3,14.0,0.757918
4,17.0,0.891960
...,...,...
95,16.0,0.181823
96,12.0,0.423721
97,14.0,0.946381
98,14.0,0.610226


In [123]:
# select 20% of consignees
_count = int(0.25 * len(set(df_train['ShipperPanjivaID'])))
Shipper_list = np.random.choice(list(set(df_train['ShipperPanjivaID'])),size=_count,replace=False)

In [124]:
len(Shipper_list)

2665

In [125]:
candidate_Consignee_list = set(df_train.loc[df_train['ShipperPanjivaID'].isin(Shipper_list) ]['ConsigneePanjivaID']) 
_count = int(0.75*len(candidate_Consignee_list))
Consignee_list = np.random.choice(list(candidate_Consignee_list) ,size = _count,replace=False)

In [126]:
tmp_df = df_test.loc[df_test['ShipperPanjivaID'].isin(Shipper_list) & df_test['ConsigneePanjivaID'].isin(Consignee_list)]

In [127]:
tmp_df

,PanjivaRecordID,Carrier,ConsigneePanjivaID,HSCode,PortOfLading,PortOfUnlading,ShipmentDestination,ShipmentOrigin,ShipperPanjivaID
11,113741055,522,2465,37,14,31,74,82,10613
15,114032447,38,6139,51,65,31,25,82,8294
20,113918719,104,7804,51,158,7,32,82,6076
28,114050495,589,2444,22,103,31,78,82,8177
33,114447295,639,4760,82,103,31,74,82,4504
...,...,...,...,...,...,...,...,...,...
40599,117362316,138,5212,61,117,53,85,89,10631
40603,117182796,492,4928,51,140,31,74,89,4974
40605,117307660,278,6684,23,68,2,44,83,4662
40611,116991564,573,3038,62,125,29,72,17,9534


In [128]:
_count = int(len(set(tmp_df['PortOfLading'])) * 0.5)
PortOfLading_list = np.random.choice(list(set(tmp_df['PortOfLading'])) ,size = _count,replace=False)

len(PortOfLading_list)

66

In [129]:
tmp_df_2 = tmp_df.loc[tmp_df['PortOfLading'].isin(PortOfLading_list)]

In [130]:
tmp_df_2

,PanjivaRecordID,Carrier,ConsigneePanjivaID,HSCode,PortOfLading,PortOfUnlading,ShipmentDestination,ShipmentOrigin,ShipperPanjivaID
15,114032447,38,6139,51,65,31,25,82,8294
20,113918719,104,7804,51,158,7,32,82,6076
28,114050495,589,2444,22,103,31,78,82,8177
33,114447295,639,4760,82,103,31,74,82,4504
36,114481151,226,3313,4,103,46,16,82,1549
...,...,...,...,...,...,...,...,...,...
40573,116589452,489,1732,51,103,31,72,83,7395
40587,116853516,320,2472,81,200,45,120,8,1157
40589,117085836,202,2838,49,158,46,16,82,314
40599,117362316,138,5212,61,117,53,85,89,10631


In [158]:
tmp_df_3 = df_test.groupby([ 
 'ShipmentDestination',
 'ShipmentOrigin']
).size().reset_index(name='count')
tmp_df_3.sample(frac = 0.4)

,ShipmentDestination,ShipmentOrigin,count
1312,129,59,1
772,78,119,1
334,33,1,5
740,78,23,1
812,79,93,2
...,...,...,...
809,79,85,3
480,57,1,1
665,73,82,4
779,79,14,10


In [165]:
tmp_df_4 = df_test.groupby([ 
 'HSCode',
 'ShipmentOrigin']
).size().reset_index(name='count')
ub = np.percentile(list(tmp_df_4['count']) , 90 )
lb = np.percentile(list(tmp_df_4['count']) , 10)
tmp_df_4.loc[(tmp_df_4['count']>= lb) & (tmp_df_4['count']<=ub)].sample(frac = 0.25)

,HSCode,ShipmentOrigin,count
714,56,48,9
226,22,0,13
1050,86,66,3
397,34,83,2
991,80,82,3
...,...,...,...
969,76,60,1
363,28,83,4
786,62,7,1
182,19,4,3


In [141]:
res1 = tmp_df.groupby(['PortOfLading','ShipmentOrigin']).size().reset_index(name='count').sort_values(by=['count'])
res2 = res1.loc[(res1['count'] >=  np.percentile(list(res1['count']),50) ) & (res1['count'] <= np.percentile(list(res1['count']),90))]

In [142]:
list_PortOfLading = list(set(res2['PortOfLading']))
list_ShipmentOrigin = list(set(res2['ShipmentOrigin']))

In [143]:
len(list_PortOfLading),len(list_ShipmentOrigin)

(91, 46)

In [144]:
len(set(df_train['PortOfLading']))

286

In [145]:
res2

,PortOfLading,ShipmentOrigin,count
234,141,4,3
293,163,89,3
355,232,82,3
216,133,103,3
27,34,42,3
...,...,...,...
24,30,52,28
228,140,83,28
42,43,83,29
119,80,103,29
